In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
import importlib.util; print(importlib.util.find_spec('transformers').origin)


expert order by l1

In [ ]:
import pickle
l1_result_bf16 = pickle.load(open("./qw2.7B.l1.bf16.pkl", "rb"))
l1_result_fp32 = pickle.load(open("./qw2.7B.l1.fp32.pkl", "rb"))

In [ ]:
l1_result_fp32

In [ ]:
values = []
for key, value in l1_result_fp32.items():
    values.append((key[0], key[1], key[2], value))
import pandas as pd
df = pd.DataFrame(values, columns=["layer", "expert", "var", "l1"])
df

In [ ]:
df = df[df["var"]=="down"]
expert_order_output = []
for layer_id in range(24):
    print("layer {}".format(layer_id))
    tmp_df = df[df["layer"]==layer_id]
    tmp_df = tmp_df.sort_values("l1")  # l1 小到大
    expert_idxs = list(tmp_df["expert"])
    l1_list = list(tmp_df["l1"])
    print(expert_idxs)
    print(l1_list)
    print("min l1 {}; max l1 {}".format(min(l1_list), max(l1_list)))
    expert_order_output.append(expert_idxs)

In [ ]:
layer_idx_to_expert_idx = {}
for i, expert_idxs in enumerate(expert_order_output):
    layer_idx_to_expert_idx[i] = expert_idxs
print(layer_idx_to_expert_idx)
import json
json.dump(layer_idx_to_expert_idx, open("./moe_prune/layer_idx_to_expert_idx.json", 'w'))

expert order by alpha

In [ ]:
import json
layer_to_alpha = json.load(open("data/layer_idx_to_alpha_list.json", 'r'))

In [ ]:
layer_to_alpha

In [ ]:
layer_idx_to_expert_idx_alpha = {}
for layer_idx in layer_to_alpha:
    alphas = layer_to_alpha[layer_idx]
    expert_idx_with_alpha = list(enumerate(alphas))
    expert_idx_with_alpha = sorted(expert_idx_with_alpha, key=lambda x: x[1], reverse=True) # alpha大到小
    expert_idx_list = list(map(lambda x: x[0], expert_idx_with_alpha))
    # print(expert_idx_list)
    layer_idx_to_expert_idx_alpha[layer_idx] = expert_idx_list
layer_idx_to_expert_idx_alpha

In [ ]:
json.dump(layer_idx_to_expert_idx_alpha, open("layer_idx_to_expert_idx.alpha.json", 'w'))

expert order by distribution

In [ ]:
import json
distribution_dic = json.load(open("../deepseek_model/route_distribution.json", 'r'))
distribution_dic

In [ ]:
layer_idx_to_expert_idx = {}
for key, value in distribution_dic.items():
    layer_idx = int(key.split("-")[0])
    expert_idx = int(key.split("-")[1])
    freq = value
    if layer_idx in layer_idx_to_expert_idx:
        layer_idx_to_expert_idx[layer_idx].append((expert_idx, freq))
    else:
        layer_idx_to_expert_idx[layer_idx] = []
        layer_idx_to_expert_idx[layer_idx].append((expert_idx, freq))

for layer_idx in layer_idx_to_expert_idx:
    expert_with_freq = layer_idx_to_expert_idx[layer_idx]
    expert_with_freq.sort(key=lambda x: x[1], reverse=True)
    experts = list(map(lambda x: x[0], expert_with_freq))
    layer_idx_to_expert_idx[layer_idx] = experts

In [5]:
json.dump(layer_idx_to_expert_idx, open("../deepseek_model/layer_idx_to_expert_idx.distribution.json", 'w'))

route ana

In [ ]:
with open("route_ana.result", 'r') as f:
    content = f.read()
tmp = eval(content)

In [ ]:
import pandas as pd
df = pd.DataFrame(index=list(range(32)), columns=range(64))

# 将数据填充到DataFrame中
for key, value in tmp.items():
    row, col = key[0], key[1]
    df.at[row, col] = value

# 将NaN值替换为0（如果需要）
df.fillna(0, inplace=True)
df.to_excel("route_ana.xlsx")